In [1]:
import data.data_source as data_source

ds = data_source.get_data_source()

DROP_SQL = "DROP TABLE IF EXISTS rate_cone_diagnostics;"
CREATE_SQL = """
-- Table: rate_cone_diagnostics
CREATE TABLE rate_cone_diagnostics (
    id               BIGSERIAL PRIMARY KEY,
    curve_type       TEXT        NOT NULL,
    model_type       TEXT        NOT NULL,
    curve_date       DATE        NOT NULL,
    days_forward     INTEGER     NOT NULL,
    tenor_num        NUMERIC     NOT NULL,
    tenor_str        TEXT        NOT NULL,
    -- Forecast distribution quantiles:
    forecast_p01     NUMERIC,
    forecast_p05     NUMERIC,
    forecast_p10     NUMERIC,
    forecast_p50     NUMERIC,
    forecast_p90     NUMERIC,
    forecast_p95     NUMERIC,
    forecast_p99     NUMERIC,
    -- Realized outcome:
    realized_date    DATE,
    realized_rate    NUMERIC,
    -- Error metrics:
    forecast_error   NUMERIC,  -- forecast_p50 - realized_rate
    absolute_error   NUMERIC,  -- abs(forecast_p50 - realized_rate)
    relative_error   NUMERIC,  -- (forecast_p50 - realized_rate) / realized_rate
    percentile_rank  NUMERIC,  -- where realized_rate falls in the forecast CDF
    inside_cone      BOOLEAN,  -- true if realized_rate between forecast_p01 and forecast_p99
    -- Fit diagnostics:
    n_obs_fit        INTEGER,  -- number of observations used to fit the cone
    total_variance   NUMERIC,  -- total variance of fit
    trace_covariance NUMERIC,  -- trace of the covariance matrix
    inserted_at      TIMESTAMPTZ NOT NULL DEFAULT now(),
    -- ensure uniqueness per model/curve/day/tenor
    UNIQUE (model_type, curve_date, days_forward, tenor_num)
);"""
CREATE_INDEX_A = """
-- Indexes for faster lookups
CREATE INDEX idx_icm_model_curve_days
  ON rate_cone_diagnostics(model_type, curve_date, days_forward);
"""
CREATE_INDEX_B = """
CREATE INDEX idx_icm_tenor
  ON rate_cone_diagnostics(tenor_num);
"""

def setup_diagnostics_table():
    ds.query(DROP_SQL)
    ds.query(CREATE_SQL)
    ds.query(CREATE_INDEX_A)
    ds.query(CREATE_INDEX_B)
    print("✅ rate_cone_metrics table dropped (if existed) and recreated.")

setup_diagnostics_table()


'data_env' not found in environment. Defaulting to 'sandbox' env.
setting env to sandbox data
getting data source for sandbox
✅ rate_cone_metrics table dropped (if existed) and recreated.
